In [1]:
import os
import csv
import schedule
import time
import pandas as pd
from csv import writer
from googleapiclient.discovery import build
from dotenv import load_dotenv

# Set API parameters.
load_dotenv()
API_KEY = os.getenv("API_KEY")
CHANNEL_ID = os.getenv("CHANNEL_ID")
youtube = build('youtube','v3', developerKey = API_KEY)

class ytStats:
    def __init__(self, API_KEY, CHANNEL_ID):
        self.API_KEY = API_KEY
        self.CHANNEL_ID = CHANNEL_ID

    def get_channel_statistics(self):
        all_data = []
        request = youtube.channels().list(
            part = "snippet,contentDetails,statistics",
            id = CHANNEL_ID
            )
        response = request.execute()

        for i in range (len(response['items'])):
            data = dict(
                result_channel_name = response['items'][i]['snippet']['title']  , 
                result_playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'] ,
                result_channel_subscribers = response['items'][i]['statistics']['subscriberCount']  , 
                result_channel_views = response['items'][i]['statistics']['viewCount']  , 
                result_channel_total_videos = response['items'][i]['statistics']['videoCount'] 
                )
            all_data.append(data)              
        return all_data

    def get_video_ids(youtube, playlist_id):
        request = youtube.playlistItems().list(
            part = 'contentDetails' , 
            playlistId = playlist_id , 
            maxResults = 50
        )
        response = request.execute()

        all_data = []

        for i in range(len(response['items'])):
            all_data.append(response['items'][i]['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')
        more_pages = True

        while more_pages:
            if next_page_token is None:
                more_pages = False
            else:
                request = youtube.playlistItems().list(
                    part = 'contentDetails' , 
                    playlistId = playlist_id , 
                    maxResults = 50 , 
                    pageToken = next_page_token
                )
                response = request.execute()
                
                for i in range(len(response['items'])):
                    all_data.append(response['items'][i]['contentDetails']['videoId'])
                
                next_page_token = response.get('nextPageToken')
        
        return all_data

    def get_video_details(youtube, video_ids):
        all_data = []
        for i in range(0, len(video_ids), 50):
            request = youtube.videos().list(
                part = 'snippet, statistics' , 
                id = ','.join(video_ids[i: i + 50]) ,
                #order = 'date' # DOES NOT WORK
            )
            response = request.execute()

            for video in response['items']:
                video_stats = dict(
                    result_video_name = video['snippet']['title']  ,
                    result_video_id = video['id'] ,                
                    result_video_description = video['snippet']['description']  ,
                    result_video_upload_time = video['snippet']['publishedAt']  ,
                    result_video_views = video['statistics']['viewCount'] ,  
                    result_video_likes = video['statistics']['likeCount'] ,  
                    result_video_favorites = video['statistics']['favoriteCount'] ,  
                    result_video_comments = video['statistics']['commentCount']  
                    )
                all_data.append(video_stats)

        return all_data

In [2]:
# Call functions
yt = ytStats
channel_stats = yt.get_channel_statistics(youtube)#, channel_id)
playlist_id    = channel_stats[0]['result_playlist_id']
video_ids      = yt.get_video_ids(youtube, playlist_id)                # Time = 5.6s
video_details  = yt.get_video_details(youtube, video_ids)              # Time = 8.5s

In [6]:
#class ytOutput:
def create_data_frame(channel_stats, video_details):
    # Create data frames.
    initial_channel_data = pd.DataFrame(channel_stats, columns = [
        'result_playlist_id' ,
        'result_channel_name' , 
        'result_channel_views' ,
        'result_channel_subscribers' ,
        'result_channel_total_videos' 
        ])
    initial_video_data = pd.DataFrame(video_details, columns = [
        'result_video_id' ,
        'result_video_upload_time' , 
        'result_video_name' , 
        'result_video_description' , 
        'result_video_views' , 
        'result_video_likes' , 
        'result_video_comments' 
        ])
    sort = initial_video_data.sort_values('result_video_upload_time', axis = 0, ascending = False)

    # Format data frames for channel_data.

    # Fromat data frames for initial_video_data.
    upload_time = initial_video_data.groupby('result_video_upload_time', as_index = False).size()
    published_dttm = pd.to_datetime(upload_time['result_video_upload_time']).dt    
    initial_video_data['published_date'] = upload_time['Dates'] = published_dttm.date
    initial_video_data['published_time'] = upload_time['Time']  = published_dttm.time
    del initial_video_data['result_video_upload_time']
    video_data = initial_video_data.reindex(columns = ['result_video_id', 'published_date', 'published_time', 'result_video_name','result_video_description','result_video_views','result_video_likes','result_video_comments'])
    video_data.columns = ['Video ID', 'Date', 'Time', 'Video Name', 'Video Description', 'Views', 'Likes', 'Comments', ]
    
    return initial_channel_data, video_data, sort

channel_data, video_data, sort = create_data_frame(channel_stats, video_details)
#sort = pd.DataFrame.sort_values(by = 'result_video_upload_time', axis=0, ascending=True, kind='mergesort')

In [7]:
sort # 0 = 42172

,result_video_id,result_video_upload_time,result_video_name,result_video_description,result_video_views,result_video_likes,result_video_comments
0,WIHknnyIVlU,2022-12-12T21:10:11Z,When you slice completely through earth,It's Slice It All!\n• Coffee - https://florida...,45864,3813,263
1,QlA8J3elpYg,2022-12-11T22:30:10Z,When a bullet shoots through the earth,It's GunSpin.\n• Coffee - https://floridamanco...,461847,16858,993
2,aGnPpmTmfaw,2022-12-10T21:26:02Z,Ending crime by ending humanity in GTA 5,It's GTA 5.\n• Subscribe - http://bit.ly/1PG8z...,306806,12759,596
3,Sswi4XdNOSY,2022-12-09T21:00:13Z,"When you evolve every species for 1,000,000 years",It's Monsters Lab - Freaky Running.\n• Coffee ...,285770,10016,364
4,p9oAWuxkTCA,2022-12-07T21:35:01Z,I scammed the biggest criminals,It's Gas Station INC.\n• Coffee - https://flor...,677476,19698,594
...,...,...,...,...,...,...,...
2921,hYeHoLZLq3A,2016-02-02T02:11:11Z,The Forest v0.31b | Part 3 | Building the Base...,No time for BS! We've got a massive amount of...,12752,612,53
2922,5pY8O_8CTSs,2016-02-01T02:18:38Z,Increase Youtube Video Game Recording Quality ...,"Just like so many others, I tried so many diff...",11651,483,77
2923,M3OoahLR9Ls,2016-01-31T23:34:56Z,Slime Rancher | Part 1 | Plorts for the Slime ...,"Today we're checking out some of the freaky, a...",49637,1604,196
2924,5hRWYVzvaHc,2016-01-31T00:56:16Z,The Forest v0.31b | Part 2 | Place for the Bas...,Beyond the plants that make energy mix and med...,19549,888,81


In [ ]:
import pandas as pd
data = pd.DataFrame([[3,0,1], [4,4,4], [1,7,7], [10,10,10]],
    index=['Python', 'Java', 'C','Kotlin'],
    columns=['RATE','EE','AA'])

sort = data.sort_values("RATE", axis = 0, ascending = False)

print("Data before sorting:\n")
print(data)

print("Data after sorting:\n")
print(sort)